In [ ]:
# creation: in each window, add a sine function of specified frequency and phase (fix frequency? or learn it?)

# experiment: do uniform phase and steup sine as you expect it to be
# for each example learn phase overall?
# learn ConvRevNet to transform the stuff afterwards, see what it does (possibly demean and desquare data before per block or sth 
## or moving...)
# also see if you learn phase per block



In [ ]:
%%capture
import os
import site
os.sys.path.insert(0, '/home/schirrmr/code/reversible/reversible2/')
os.sys.path.insert(0, '/home/schirrmr/braindecode/code/braindecode/')
os.sys.path.insert(0, '/home/schirrmr/code/explaining/reversible//')
%cd /home/schirrmr/


%load_ext autoreload
%autoreload 2
import numpy as np
import logging
log = logging.getLogger()
log.setLevel('INFO')
import sys
logging.basicConfig(format='%(asctime)s %(levelname)s : %(message)s',
                     level=logging.INFO, stream=sys.stdout)
import matplotlib
from matplotlib import pyplot as plt
from matplotlib import cm
%matplotlib inline
%config InlineBackend.figure_format = 'png'
matplotlib.rcParams['figure.figsize'] = (12.0, 1.0)
matplotlib.rcParams['font.size'] = 14
from matplotlib import rcParams, cycler
import seaborn
seaborn.set_style('darkgrid')

from reversible.sliced import sliced_from_samples

from numpy.random import RandomState

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
import numpy as np
import copy
import math

import itertools
from reversible.plot import create_bw_image
import torch as th
from braindecode.torch_ext.util import np_to_var, var_to_np
from reversible.revnet import ResidualBlock, invert, SubsampleSplitter, ViewAs, ReversibleBlockOld
from spectral_norm import spectral_norm
from conv_spectral_norm import conv_spectral_norm

def display_text(text, fontsize=18):
    fig = plt.figure(figsize=(12,0.1))
    plt.title(text, fontsize=fontsize)
    plt.axis('off')
    display(fig)
    plt.close(fig)

In [ ]:
from braindecode.datasets.bbci import BBCIDataset
from braindecode.mne_ext.signalproc import mne_apply
# we loaded all sensors to always get same cleaning results independent of sensor selection
# There is an inbuilt heuristic that tries to use only EEG channels and that definitely
# works for datasets in our paper
#train_loader = BBCIDataset('/data/schirrmr/schirrmr/HGD-public/reduced/train/13.mat')
#test_loader = BBCIDataset('/data/schirrmr/schirrmr/HGD-public/reduced/test/13.mat')
start_cnt = BBCIDataset('/data/schirrmr/schirrmr/HGD-public/reduced/train/4.mat',).load()
start_cnt = start_cnt.drop_channels(['STI 014'])
def car(a):
    return a - np.mean(a, keepdims=True, axis=0)

start_cnt = mne_apply(
    car, start_cnt)

start_cnt = start_cnt.reorder_channels(['C3', 'C4'])

In [ ]:
from collections import OrderedDict
from braindecode.datautil.trial_segment import create_signal_target_from_raw_mne

marker_def = OrderedDict([('Right Hand', [1]), ('Left Hand', [2],),
                         ('Rest', [3]), ('Feet', [4])])
ival = [500,1500]
from braindecode.mne_ext.signalproc import mne_apply, resample_cnt
from braindecode.datautil.signalproc import exponential_running_standardize, bandpass_cnt

log.info("Resampling train...")
cnt = resample_cnt(start_cnt, 250.0)
log.info("Standardizing train...")
cnt = mne_apply(lambda a: exponential_running_standardize(a.T ,factor_new=1e-3, init_block_size=1000, eps=1e-4).T,
                     cnt)
cnt = resample_cnt(cnt, 32.0)
cnt = resample_cnt(cnt, 64.0)
#cnt = mne_apply(
#    lambda a: bandpass_cnt(a, 0, 2, cnt.info['sfreq'],
#                           filt_order=10,
#                           axis=1), cnt)

train_set = create_signal_target_from_raw_mne(cnt, marker_def, ival)
cnt_bandpassed =  mne_apply(
    lambda a: bandpass_cnt(a, 8, 13, cnt.info['sfreq'],
                           filt_order=10,
                           axis=1), cnt)
alpha_set = create_signal_target_from_raw_mne(cnt_bandpassed, marker_def, ival)
x_alpha_right = alpha_set.X[alpha_set.y == 0]

x_alpha_rest = alpha_set.X[alpha_set.y == 2]

alpha_a = np_to_var(x_alpha_right[:160,0:1,:,None], dtype=np.float32)

alpha_b = np_to_var(x_alpha_rest[:160,0:1,:,None], dtype=np.float32)
inputs_alpha = [alpha_a, alpha_b]


In [ ]:
i_class = 0
a_in = inputs_alpha[i_class][0]
plt.plot(var_to_np(a_in).squeeze())

In [ ]:
from reversible.util import set_random_seeds

set_random_seeds(3, True)

learned_phase = th.zeros(1,requires_grad=True)
phases = th.ones(8) * learned_phase
example = [th.sin(th.linspace(0,2*np.pi,9)[:8] + p) for p in phases]
plt.plot(var_to_np(th.cat(example)))

In [ ]:
optimizer = th.optim.Adam([learned_phase], lr=1e-2)#, coef_a, coef_b
n_epochs = 1000
for i_epoch in range(n_epochs):
    phases = th.ones(8) * learned_phase
    example = [th.sin(th.linspace(0,2*np.pi,9)[:8] + p) for p in phases]
    loss = th.sqrt(th.sum((th.cat(example) - a_in.squeeze()) ** 2))
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    if i_epoch % (n_epochs // 10) == 0:
        print("Loss: {:.4f}".format(loss.item()))
        print("Phase: {:.2f}".format(learned_phase.item()))
        fig = plt.figure(figsize=(12,2))
        plt.plot(var_to_np(a_in).squeeze())
        plt.plot(var_to_np(th.cat(example)))
        display(fig)
        plt.close(fig)

In [ ]:
nn.functional.adaptive_avg_pool1d()

In [ ]:
example = th.sin(th.linspace(0,2*np.pi,9)[:8]).repeat(8,1)
plt.plot(var_to_np(example.view(-1)))

### now with normed inputs

In [ ]:
in_squared = th.sqrt(nn.functional.avg_pool2d(inputs_alpha[i_class] ** 2,(7,1),stride=(1,1),padding=(3,0)) * 7)

in_normed = inputs_alpha[i_class] / in_squared

b_in = in_normed[0]

In [ ]:

set_random_seeds(3, True)
learned_phase = th.zeros(1,requires_grad=True)
phases = th.ones(8) * learned_phase
example = [th.sin(th.linspace(0,2*np.pi,9)[:8] + p) for p in phases]
plt.plot(var_to_np(th.cat(example)))

In [ ]:
optimizer = th.optim.Adam([learned_phase], lr=1e-2)#, coef_a, coef_b
n_epochs = 1000
for i_epoch in range(n_epochs):
    phases = th.ones(8) * learned_phase
    example = [th.sin(th.linspace(0,2*np.pi,9)[:8] + p) for p in phases]
    loss = th.sqrt(th.sum((th.cat(example) - b_in.squeeze()) ** 2))
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    if i_epoch % (n_epochs // 10) == 0:
        print("Loss: {:.4f}".format(loss.item()))
        print("Phase: {:.2f}".format(learned_phase.item()))
        fig = plt.figure(figsize=(12,2))
        plt.plot(var_to_np(b_in).squeeze())
        plt.plot(var_to_np(th.cat(example)))
        display(fig)
        plt.close(fig)

### learned phase per block

In [ ]:
set_random_seeds(3, True)
learned_phase = th.zeros(8,requires_grad=True)
phases = learned_phase
example = [th.sin(th.linspace(0,2*np.pi,9)[:8] + p) for p in phases]
plt.plot(var_to_np(th.cat(example)))

In [ ]:
optimizer = th.optim.Adam([learned_phase], lr=1e-2)#, coef_a, coef_b
n_epochs = 1000
for i_epoch in range(n_epochs):
    phases = learned_phase
    example = [th.sin(th.linspace(0,2*np.pi,9)[:8] + p) for p in phases]
    loss = th.sqrt(th.sum((th.cat(example) - b_in.squeeze()) ** 2))
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    if i_epoch % (n_epochs // 10) == 0:
        print("Loss: {:.4f}".format(loss.item()))
        print("Phase: {:s}".format(str(learned_phase)))
        fig = plt.figure(figsize=(12,2))
        plt.plot(var_to_np(b_in).squeeze())
        plt.plot(var_to_np(th.cat(example)))
        display(fig)
        plt.close(fig)

In [ ]:
import scipy

In [ ]:
signal = var_to_np(a_in).squeeze()
hilb = scipy.signal.hilbert(signal)

In [ ]:
plt.plot(signal)
plt.plot(np.real(hilb))
plt.plot(np.imag(hilb))

In [ ]:
signal = np.sin(np.linspace(0,64*np.pi,64))
hilb = scipy.signal.hilbert(signal)
plt.plot(signal)
plt.plot(np.real(hilb))
plt.plot(np.imag(hilb))

## learned phase frequency and amplitude per block

In [ ]:
set_random_seeds(3, True)
learned_phases = th.zeros(8,requires_grad=True)
learned_freqs =  th.ones(8,requires_grad=True)
learned_freqs.data[:] = 2
learned_amps =  th.ones(8,requires_grad=True)
example = [th.sin(th.linspace(0,np.pi,9)[:8] * f + p) * a
           for f,a,p in zip(learned_freqs, learned_amps, learned_phases)]
plt.plot(var_to_np(th.cat(example)))

In [ ]:
optimizer = th.optim.Adam([learned_freqs, learned_amps, learned_phases,], lr=1e-2)#, coef_a, coef_b
n_epochs = 10000
for i_epoch in range(n_epochs):
    phases = learned_phase
    example = [th.sin(th.linspace(0, np.pi,9)[:8] * f + p) * a
               for f,a,p in zip(learned_freqs, learned_amps, learned_phases)]
    loss = th.sqrt(th.sum((th.cat(example) - b_in.squeeze()) ** 2))
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    if i_epoch % (n_epochs // 10) == 0:
        print("Loss: {:.4f}".format(loss.item()))
        print("Phase: {:s}".format(str(learned_phase)))
        fig = plt.figure(figsize=(12,2))
        plt.plot(var_to_np(b_in).squeeze())
        plt.plot(var_to_np(th.cat(example)))
        display(fig)
        plt.close(fig)

In [ ]:
th.mean(learned_freqs) * 4

In [ ]:
np.mean([2.5973,  2.4428, 2.7306,  2.6816, 2.9168],)

In [ ]:
(2.67 * 8) / 2

In [ ]:
learned_freqs

In [ ]:
# What we want: model that can create repetitive patterns easily
# We see that often it seems to not model it "nicely" onto a single dimension
# Also not easy to interpret parts
# 

# a) in each window, transform the pattern


In [ ]:
# creation: in each window, add a sine function of specified frequency and phase (fix frequency? or learn it?)

# experiment: do uniform phase and steup sine as you expect it to be
# for each example learn phase overall?
# learn ConvRevNet to transform the stuff afterwards, see what it does (possibly demean and desquare data before per block or sth 
## or moving...)
# also see if you learn phase per block

